**SINGLE DOWNLOAD**

In [ ]:
import nest_asyncio
import asyncio
from telethon import TelegramClient

nest_asyncio.apply()

# 🔒 Enter your data:
api_id = ...      # insert your API ID
api_hash = '...'  # insert your API hash
username = '...'  # insert telegram username without "@"

client = TelegramClient('session_name', api_id, api_hash)

async def get_user_id():
    await client.start()
    try:
        user = await client.get_entity(username)
        print(f"Username: @{user.username}")
        print(f"ID: {user.id}")
        print(f"Name: {user.first_name}")
        print(f"Object_type: {type(user)}")
    except Exception as e:
        print("Error:", e)
    await client.disconnect()

await get_user_id()

**MULTI DOWNLOAD**

In [ ]:
import nest_asyncio
import asyncio
from telethon import TelegramClient
import pandas as pd

nest_asyncio.apply()

# 🔒 Enter your data:
api_id = ...      # insert your API ID
api_hash = '...'  # insert your API hash

# insert telegram usernames without "@"
usernames = ["username1", "username2", "username3", etc.]

client = TelegramClient('session_name', api_id, api_hash)

async def get_users_info():
    await client.start()
    results = []

    for username in usernames:
        try:
            user = await client.get_entity(username)
            
            results.append({
                "username": f"@{user.username}" if user.username else None,
                "id": user.id,
                "first_name": user.first_name,
                "last_name": user.last_name,
                "phone": user.phone if hasattr(user, 'phone') else None,
            })
        except Exception as e:
            results.append({
                "username": f"@{username}",
                "id": None,
                "first_name": None,
                "last_name": None,
                "phone": None,
                "error": str(e)
            })

    await client.disconnect()
    return results

results = asyncio.get_event_loop().run_until_complete(get_users_info())

# Convert to DataFrame and save to CSV
df = pd.DataFrame(results)
df.to_csv("tg_full_info.csv", index=False, encoding="utf-8-sig")

print("File tg_full_info.csv successfully saved!")
print(df)

**GROUP DOWNLOAD**

In [ ]:
import os
import time
import nest_asyncio
import asyncio
import pandas as pd
from urllib.parse import urlparse
from telethon import TelegramClient, errors
from telethon.sessions import MemorySession

nest_asyncio.apply()

# 🔒 Enter your data:
api_id = ...      # insert your API ID
api_hash = '...'  # insert your API hash

# Chat link
chat_link = "https://t.me/chat_name" # insert here your chat link in the same format

chat_name = os.path.basename(urlparse(chat_link).path)
csv_filename = f"{chat_name}_chat_members.csv"

client = TelegramClient(MemorySession(), api_id, api_hash)

async def dump_members():
    await client.start()

    data = []
    saved_ids = set()
    if os.path.exists(csv_filename) and os.path.getsize(csv_filename) > 0:
        df_old = pd.read_csv(csv_filename)
        if "id" in df_old.columns:
            saved_ids = set(df_old["id"].tolist())
            data = df_old.to_dict("records")
            print(f"🔄 Participants found {len(saved_ids)} in existing file, continuing upload...")
    else:
        print(f"📂 {csv_filename} empty or does not exist, we start from scratch...")

    try:
        async for user in client.iter_participants(chat_link):
            if user.id not in saved_ids:
                data.append({
                    "id": user.id,
                    "username": f"@{user.username}" if user.username else None,
                    "first_name": user.first_name,
                    "last_name": user.last_name,
                    "phone": user.phone
                })
                saved_ids.add(user.id)

                if len(data) % 50 == 0:
                    pd.DataFrame(data).to_csv(csv_filename, index=False, encoding="utf-8-sig")

    except errors.FloodWaitError as e:
        print(f"⏳ FloodWait: wait for {e.seconds} sec...")
        time.sleep(e.seconds)
    except Exception as e:
        print(f"⚠️ Ошибка: {e}")

    pd.DataFrame(data).to_csv(csv_filename, index=False, encoding="utf-8-sig")
    print(f"✅ Saved {len(data)} participants in {csv_filename}")

asyncio.get_event_loop().run_until_complete(dump_members())